In [74]:
import numpy as np
import pandas as pd
import os, sys

In [75]:
### set up paths, etc. 
path_to_stim = os.getcwd()

## load in run5_submitButton data
M = pd.read_csv(os.path.join(path_to_stim,'graphical_conventions_group_data_run5_submitButton.json'))

## TODO: add several columns to dataframe above: 'target_url', 'basic', 'subset', 'cluster','width','height' column based on dictionaries here: https://github.com/cogtoolslab/semantic_mapping/issues/1
shapenet_30afd2ef2ed30238aa3d0a2f00b54836 = {'filename': "30afd2ef2ed30238aa3d0a2f00b54836.png" , 'basic': "dining", 'subordinate': "dining_00" , 'subset':"A", 'cluster': 1, 'object': 0, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/30afd2ef2ed30238aa3d0a2f00b54836.png",'width': 256, 'height': 256};
shapenet_30dc9d9cfbc01e19950c1f85d919ebc2 = {'filename': "30dc9d9cfbc01e19950c1f85d919ebc2.png" , 'basic': "dining", 'subordinate': "dining_01" , 'subset':"A", 'cluster': 1, 'object': 1, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/30dc9d9cfbc01e19950c1f85d919ebc2.png",'width': 256, 'height': 256};
shapenet_4c1777173111f2e380a88936375f2ef4 = {'filename': "4c1777173111f2e380a88936375f2ef4.png" , 'basic': "dining", 'subordinate': "dining_02" , 'subset':"B", 'cluster': 1, 'object': 2, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/4c1777173111f2e380a88936375f2ef4.png",'width': 256, 'height': 256};
shapenet_3466b6ecd040e252c215f685ba622927 = {'filename': "3466b6ecd040e252c215f685ba622927.png" , 'basic': "dining", 'subordinate': "dining_03" , 'subset':"B", 'cluster': 1, 'object': 3, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/3466b6ecd040e252c215f685ba622927.png",'width': 256, 'height': 256};
shapenet_38f87e02e850d3bd1d5ccc40b510e4bd = {'filename': "38f87e02e850d3bd1d5ccc40b510e4bd.png" , 'basic': "dining", 'subordinate': "dining_04" , 'subset':"B", 'cluster': 1, 'object': 4, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/38f87e02e850d3bd1d5ccc40b510e4bd.png",'width': 256, 'height': 256};
shapenet_3cf6db91f872d26c222659d33fd79709 = {'filename': "3cf6db91f872d26c222659d33fd79709.png" , 'basic': "dining", 'subordinate': "dining_05" , 'subset':"B", 'cluster': 1, 'object': 5, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/3cf6db91f872d26c222659d33fd79709.png",'width': 256, 'height': 256};
shapenet_3d7ebe5de86294b3f6bcd046624c43c9 = {'filename': "3d7ebe5de86294b3f6bcd046624c43c9.png" , 'basic': "dining", 'subordinate': "dining_06" , 'subset':"A", 'cluster': 1, 'object': 6, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/3d7ebe5de86294b3f6bcd046624c43c9.png",'width': 256, 'height': 256};
shapenet_56262eebe592b085d319c38340319ae4 = {'filename': "56262eebe592b085d319c38340319ae4.png" , 'basic': "dining", 'subordinate': "dining_07" , 'subset':"A", 'cluster': 1, 'object': 7, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/56262eebe592b085d319c38340319ae4.png",'width': 256, 'height': 256};
shapenet_1d1641362ad5a34ac3bd24f986301745 = {'filename': "1d1641362ad5a34ac3bd24f986301745.png" , 'basic': "waiting", 'subordinate': "waiting_00" , 'subset':"A", 'cluster': 3, 'object': 0, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/1d1641362ad5a34ac3bd24f986301745.png",'width': 256, 'height': 256};
shapenet_1da9942b2ab7082b2ba1fdc12ecb5c9e = {'filename': "1da9942b2ab7082b2ba1fdc12ecb5c9e.png" , 'basic': "waiting", 'subordinate': "waiting_01" , 'subset':"A", 'cluster': 3, 'object': 1, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/1da9942b2ab7082b2ba1fdc12ecb5c9e.png",'width': 256, 'height': 256};
shapenet_2448d9aeda5bb9b0f4b6538438a0b930 = {'filename': "2448d9aeda5bb9b0f4b6538438a0b930.png" , 'basic': "waiting", 'subordinate': "waiting_02" , 'subset':"B", 'cluster': 3, 'object': 2, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/2448d9aeda5bb9b0f4b6538438a0b930.png",'width': 256, 'height': 256};
shapenet_23b0da45f23e5fb4f4b6538438a0b930 = {'filename': "23b0da45f23e5fb4f4b6538438a0b930.png" , 'basic': "waiting", 'subordinate': "waiting_03" , 'subset':"B", 'cluster': 3, 'object': 3, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/23b0da45f23e5fb4f4b6538438a0b930.png",'width': 256, 'height': 256};
shapenet_2b5953c986dd08f2f91663a74ccd2338 = {'filename': "2b5953c986dd08f2f91663a74ccd2338.png" , 'basic': "waiting", 'subordinate': "waiting_04" , 'subset':"B", 'cluster': 3, 'object': 4, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/2b5953c986dd08f2f91663a74ccd2338.png",'width': 256, 'height': 256};
shapenet_2e291f35746e94fa62762c7262e78952 = {'filename': "2e291f35746e94fa62762c7262e78952.png" , 'basic': "waiting", 'subordinate': "waiting_05" , 'subset':"B", 'cluster': 3, 'object': 5, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/2e291f35746e94fa62762c7262e78952.png",'width': 256, 'height': 256};
shapenet_2eaab78d6e4c4f2d7b0c85d2effc7e09 = {'filename': "2eaab78d6e4c4f2d7b0c85d2effc7e09.png" , 'basic': "waiting", 'subordinate': "waiting_06" , 'subset':"A", 'cluster': 3, 'object': 6, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/2eaab78d6e4c4f2d7b0c85d2effc7e09.png",'width': 256, 'height': 256};
shapenet_309674bdec2d24d7597976c675750537 = {'filename': "309674bdec2d24d7597976c675750537.png" , 'basic': "waiting", 'subordinate': "waiting_07" , 'subset':"A", 'cluster': 3, 'object': 7, 'pose': 35, 'target_url': "https://s3.amazonaws.com/shapenet-graphical-conventions/309674bdec2d24d7597976c675750537.png",'width': 256, 'height': 256};
## subset single example gameID and repeated object (one that we have 8 unique trialNums associated with), and 
## write this out using `.to_json()` to mock up single trial prototype
get_url = {}
l = [shapenet_30afd2ef2ed30238aa3d0a2f00b54836,\
    shapenet_30dc9d9cfbc01e19950c1f85d919ebc2,\
    shapenet_4c1777173111f2e380a88936375f2ef4,\
    shapenet_3466b6ecd040e252c215f685ba622927,\
    shapenet_38f87e02e850d3bd1d5ccc40b510e4bd,\
    shapenet_3cf6db91f872d26c222659d33fd79709,\
    shapenet_3d7ebe5de86294b3f6bcd046624c43c9,\
    shapenet_56262eebe592b085d319c38340319ae4,\
    shapenet_1d1641362ad5a34ac3bd24f986301745,\
    shapenet_1da9942b2ab7082b2ba1fdc12ecb5c9e,\
    shapenet_2448d9aeda5bb9b0f4b6538438a0b930,\
    shapenet_23b0da45f23e5fb4f4b6538438a0b930,\
    shapenet_2b5953c986dd08f2f91663a74ccd2338,\
    shapenet_2e291f35746e94fa62762c7262e78952,\
    shapenet_2eaab78d6e4c4f2d7b0c85d2effc7e09,\
    shapenet_309674bdec2d24d7597976c675750537]

for row in l:
    get_url[row['subordinate']] = row['target_url']
get_url

M['target_url'] = M.apply(lambda x: get_url[x['target']], axis = 1)

In [70]:
#Sanity Check for the data and find the ones that are repeated
 #dataframe
num_freq = {}
for i, group in M.groupby(['gameID','target']):
    num = len(set(list(group.trialNum)))
    if num not in num_freq:
        num_freq[num] = 1
    else:
        num_freq[num] +=1
    
assert list(num_freq.keys()) == [2,8]
print("Sanity Check Passed!")

In [83]:
M.to_csv('./graphical_conventions_group_data_run5_submitButton_with_TargetURL.csv')
M2 = M[M['condition']=='repeated'].iloc[0].to_csv('./dummy_row.csv')

# Insert to a new DB!

In [84]:
import pymongo as pm
import pandas as pd
import json
from IPython.display import clear_output

{'control', 'repeated'}